In [2]:
from bs4 import BeautifulSoup  # For parsing HTML
import requests  # For making HTTP requests
import pandas as pd
import numpy as np

##### Scrap Individual Player Stats

In [3]:
#Get top 100 clubs
url_club = 'https://www.transfermarkt.com/vereins-statistik/wertvollstemannschaften/marktwertetop/plus/0/galerie/0?land_id=0&kontinent_id=0&yt0=Show'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
response_club= requests.get(url_club, headers=headers)
soup_club = BeautifulSoup(response_club.content, 'html.parser')

In [5]:
club_links = []
pagination_link = []
club_table = soup_club.find_all('table', class_=['items'])

for li in soup_club.find_all('li', class_=["tm-pagination__list-item"]):
    for a in li.find_all('a'):      
        pagination_link.append(a['href'])

unique_links = list(set(pagination_link))

for link in unique_links:
    page_output= requests.get(url_club, headers=headers)
    page_soup = BeautifulSoup(page_output.content, 'html.parser')
    page_table = page_soup.find_all('table', class_=['items'])

    for td in page_table[0].find_all('td', class_=["zentriert no-border-rechts"]):
        for a in td.find_all('a'):      
            club_links.append(a.get('href'))

In [73]:
base_url = 'https://www.transfermarkt.com'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

df = pd.DataFrame(columns=['#', 'Player', 'Date of birth / Age', 'Nat.', 'Height', 'Foot', 'Joined', 'Signed from', 'Contract', 'Market value'])
players_link = []

for club in club_links:
    detail_list= club.split('/')[:5]
    detail_list[2] = 'kader'
    detail_url = '/'.join(detail_list)
    detail_url = detail_url + '/plus/1/galerie/0?saison_id=2022'
    url = base_url + detail_url
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    links = soup.find_all("tr")
    data_table = soup.find_all('table', class_=['items'])

    count = 0
    player_detail = {}

    for tr in data_table[0].find_all('tr'):
        player_detail[count] = []
        for td in tr.find_all('td', class_=['zentriert', 'hauptlink']):
            if td.text.strip():
                player_detail[count].append(td.text.strip())
            for img in td.find_all('img'):
                player_detail[count].append(img['alt'].strip())
            for a in td.find_all('a'):
                players_link.append(a['href'])
        count += 1
    
    keys_to_remove = []
    for key, value in player_detail.items():
        if len(value) < 5:
            keys_to_remove.append(key)

    for key in keys_to_remove:
        del player_detail[key]
        
    for idx, data in player_detail.items():
        if len(data) == 10:    df.loc[len(df)] = data
        elif len(data) == 11: 
            data.pop(4)
            df.loc[len(df)] = data   


In [74]:
df

,#,Player,Date of birth / Age,Nat.,Height,Foot,Joined,Signed from,Contract,Market value
0,31,Ederson,"Aug 17, 1993 (29)",Brazil,"1,88m",left,"Jul 1, 2017",SL Benfica,"Jun 30, 2026",€45.00m
1,18,Stefan Ortega,"Nov 6, 1992 (30)",Germany,"1,85m",right,"Jul 1, 2022",Arminia Bielefeld,"Jun 30, 2025",€6.00m
2,-,Zack Steffen,"Apr 2, 1995 (28)",United States,"1,91m",right,"Jul 9, 2019",Columbus Crew SC,"Jun 30, 2025",€3.50m
3,33,Scott Carson,"Sep 3, 1985 (37)",England,"1,88m",right,"Jul 20, 2021",Derby County,"Jun 30, 2024",€250k
4,3,Rúben Dias,"May 14, 1997 (26)",Portugal,"1,87m",right,"Sep 29, 2020",SL Benfica,"Jun 30, 2027",€75.00m
...,...,...,...,...,...,...,...,...,...,...
2779,14,Takefusa Kubo,"Jun 4, 2001 (22)",Japan,"1,73m",left,"Jul 19, 2022",Real Madrid,"Jun 30, 2027",€15.00m
2780,25,Umar Sadiq,"Feb 2, 1997 (26)",Nigeria,"1,91m",right,"Sep 1, 2022",UD Almería,"Jun 30, 2028",€18.00m
2781,11,Mohamed-Ali Cho,"Jan 19, 2004 (19)",France,"1,81m",left,"Jul 1, 2022",Angers SCO,"Jun 30, 2027",€15.00m
2782,19,Alexander Sørloth,"Dec 5, 1995 (27)",Norway,"1,95m",left,"Aug 29, 2022",RB Leipzig,"Jun 30, 2023",€12.00m


##### Scrap Player Contract

In [76]:
for item in players_link.copy():
    if "profil" not in item:
        players_link.remove(item)
len(players_link)


5648

In [81]:
players_link_clean = list(set(players_link))

In [159]:
new_player_link = players_link_clean[0].split('/')
new_player_link[2] = 'leistungsdatendetails'
new_player_link = ('/').join(new_player_link)

player_url = base_url + new_player_link + '/saison/2022/verein/0/liga/0/wettbewerb/GB1/pos/0/trainer_id/0/plus/1'
player_response = requests.get(player_url, headers=headers)
player_soup = BeautifulSoup(player_response.content, 'html.parser')


In [160]:
stats_header = []
data_table = player_soup.find_all('table', class_=['items'])

for div in player_soup.find_all('div', class_=['data-header__details']):
    for li in div.find_all('span', class_=['data-header__content']):
        print(li.text.strip())

# for tr in data_table[0].find_all('tr'):
#     for th in tr.find_all('th'):
#         for a in th.find_all('a'):
#             if a.text.strip(): stats_header.append(a.text.strip())
#             for span in a.find_all('span'):
#                 if span['title'].strip: stats_header.append(span['title'].strip())

# for tfoot in data_table[0].find_all('tfoot'):
#     for tr in tfoot.find_all('tr'):
#         for td in tr.find_all('td', class_ =['zentriert', 'rechts']):
#             print(td.text)

Jun 17, 1990                                                                    (32)
Sunderland
England
1,87 m
Central Midfield
ROOF
England


In [132]:
# Position
stats_header

['Competition',
 'reihenfolge ASC, saison_id DESC, verein_id ASC',
 'Club',
 'Squad',
 'Appearances',
 'PPG',
 'Goals',
 'Assists',
 'Own goals',
 'Substitutions on',
 'Substitutions off',
 'Yellow cards',
 'Second yellow cards',
 'Red cards',
 'Penalty goals',
 'Minutes per goal',
 'Minutes played']